# Entorno de trabajo: Crear un dataset de noticias de prensa a partir del motor de búsqueda Sophia

Este notebook muestra una manera de extraer noticias desde el motor de búsqueda de _Sophia_, a partir de una **palabra clave**, un **país** y un **periodo de tiempo**.


_Sophia_ es un proyecto I+i+D desarrollado en la Universidad Austral de Chile desde 2019. El motor de búsqueda se encuentra en fase de mejoramiento continuo. Por cualquier pregunta o problema técnica con este notebook, no dudar en contactarme por Discord (@MatthieuVernier) o por correo mvernier@inf.uach.cl.

## 0. Librerías útiles

- Elasticsearch es el motor de búsqueda utilizado por Sophia
- Requiere la instalación del paquete elasticsearch

In [3]:
#!pip install elasticsearch

In [35]:
from elasticsearch import Elasticsearch
from datetime import datetime

## 1. Parámetros de la búsqueda

Características del motor de búsqueda:  
- El formato de las fechas es: YYYY-MM-DD
- El motor de búsqueda incluye noticias por distintos paises, pero se recomienda utilizar "chile" por el momento.
- El motor de búsqueda incluye datos entre el año 2000 y el año 2021.

In [41]:
country="chile"
from_="2018-01-01"
to_="2021-12-31"

keyword="contaminación ambiental"
simple_keyword=False

## 2. Parámetros de conexión al motor de búsqueda de Sophia

Por el momento, Sophia está accesible a través de una IP que se actualiza cada día y proximamente estará disponible en el url https://search.sophia2.org. 
No duden en pedir la IP a través de Discord si desean utilizar Sophia...

In [42]:
IP = "66.228.46.234"
PORT = 9200
USER= "elastic"
PASS= "uZCxWGE35lD3lhc"

In [43]:
#NO MODIFICAR

es = Elasticsearch(
    IP,
    port=PORT,
    http_auth=(USER, PASS)
)

## 3. Búsqueda de noticias de prensa

- Preparación de la consulta en el formato solicitado por Elasticsearch

In [44]:
#NO MODIFICAR
match=""
if (simple_keyword):
    match="match"
else:
    match="match_phrase"

query = { 
    "bool": { 
      "must": [
        {match: { "text":keyword}}

      ],
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }}},
        { "term":  { "country": country }} 
    ]
    }  
}

- Envio de la consulta
- No se puede devolver más de 10.000 noticias. En caso de obtener 10.000 respuestas, se recomienda buscar en un periodo de tiempo más corto o modificar la consulta.

In [45]:
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 524 noticias encontradas...


## 4. Guardar los datos en un dataframe Pandas y en un archivo CSV

In [46]:
import pandas as pd

data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[],'search':[]}

df = pd.DataFrame(data)  
  
for hit in res['hits']['hits']:
    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    search = keyword
    
    new_row = {'id_news':id_news, 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date, 'search':search}
    
    df = df.append(new_row, ignore_index=True)

df

,id_news,country,media_outlet,url,title,text,date,search
0,21883133.0,chile,biobiochile,https://www.biobiochile.cl/noticias/nacional/r...,Vecinos de Valdivia acusan nula respuesta tras...,Vecinos del barrio El Bosque de Valdivia esper...,2021-10-23,contaminación ambiental
1,21938703.0,chile,biobiochile,https://www.biobiochile.cl/noticias/nacional/r...,Los Lagos: advierten que mala calidad de aire ...,Desde el departamento de Epidemiología de la s...,2021-06-22,contaminación ambiental
2,367985.0,chile,horas24,https://www.24horas.cl/nacional/decretan-alert...,Decretan Alerta Amarilla para Quintero y Puchu...,Dado el nuevo episodio de contaminación ambien...,2018-08-23,contaminación ambiental
3,5198643.0,chile,ahoranoticiasmega,https://www.meganoticias.cl/nacional/265471-co...,Toxicólogo llama a no realizar deporte en epis...,Los informes son lapidarios. Según la Organiza...,2019-06-20,contaminación ambiental
4,353090.0,chile,biobiochile,https://www.biobiochile.cl/noticias/nacional/r...,La Araucanía: investigadores mapuches e INDH o...,Investigadores mapuches ofician a través del I...,2020-04-05,contaminación ambiental
...,...,...,...,...,...,...,...,...
519,5162242.0,chile,elciudadano,https://www.elciudadano.com/medio-ambiente/min...,Minera BioLantánidos: El proyecto de extracció...,El proyecto de la minera BioLantánidos se pres...,2020-10-08,contaminación ambiental
520,21958809.0,chile,elciudadano,https://www.elciudadano.com/mexico/2021-atoyac...,2021: Atoyac y el ecocidio que no se detiene,"La polución de la cuenca Atoyac-Zahuapan, cons...",2021-04-23,contaminación ambiental
521,5172828.0,chile,elciudadano,https://www.elciudadano.com/medio-ambiente/pro...,"Proyecto Vizcachitas, el ecocidio minero que f...",Por Francisco Marín Castro La reciente aprobac...,2020-06-08,contaminación ambiental
522,6208561.0,chile,latercera,https://www.latercera.com/tendencias/noticia/r...,Raúl Sohr: “El desabastecimiento de agua puede...,El periodista Raúl Sohr está camiseteado con l...,2020-10-30,contaminación ambiental


- Verifiquemos si existen noticias con la misma URL

In [47]:
df['url'].duplicated().any()

False

- Eliminar las noticias duplicadas

In [48]:
df = df.drop_duplicates(subset='url', keep='first')

- Guardar los archivos en un CSV

In [49]:
nombre_archivo=keyword.replace(' ', '')+"_"+country+"_"+from_+"_"+to_+".csv"
print("El archivo se llama:")
print(nombre_archivo)
df.to_csv("./"+nombre_archivo)

El archivo se llama:
contaminaciónambiental_chile_2018-01-01_2021-12-31.csv
